In [ ]:
nds=['Rayman DS (E)','Mario Kart DS (E)(Spluff)','Mario & Luigi - Partners in Time (E)(Trashman)','Phoenix Wright -Ace Attorney- (E)','New Super Mario Bros. (U)(pSyDS)','New Super Mario Brothers (E)(Supremacy)','Viewtiful Joe - Double Trouble! (S)(Wee Team)','42 All-Time Classics (E)(Legacy)','Jump Ultimate Stars [JAP+ESP]','Phoenix Wright - Ace Attorney (EU)(M3)','Phoenix Wright - Ace Attorney (Europe) (De,Es,It)','Phoenix Wright Ace Attorney (E)(Supremacy)','Vegas Casino High 5! (E)(Legacy)','Rayman Raving Rabbids (EU)(M6)','Phoenix Wright Ace Attorney - Justice For All (E)','Phoenix Wright - Ace Attorney - Justice for All (EU)(M2)','Phoenix Wright Ace Attorney Justice For All (E)','0926 Phoenix Wright Ace Attorney -Justice For All- (E)','0965 - Mind Quiz - Your Brain Coach (E)(Supremacy)','1000 - Hotel Dusk - Room 215 (E)(Supremacy)','101 juegos en 1','Kirby - Mouse Attack (E)(XenoPhobia)','Mouse Attack (E)(XenoPhobia).RTS','Chessmaster - The Art of Learning (E)(EXiMiUS)','Simpsons El Videojuego, Los (S)(EXiMiUS)','Cooking Mama READ NFO','Mario Party DS (EU)(M5)','1997 - Mario & Sonic at the Olympic )','Cooking Mama 2 - Dinner with Friends','Bomberman Land Touch! 2 (E)(XenoPhobia)','Apollo Justice - Ace Attorney (EU)(M2)','My Health Coach - Manage Your Weight','Unsolved Crimes (E)(XenoPhobia)','Time Hollow (E)(EXiMiUS)','Bleach - DaiUS)','Travel Games for Dummies (EU)(M5)(XenoPhobia)','Jake Hunter Detective Story  t','Date ni Gametsui Wake ja Nee! - Dungeon Maker Girls Type (Japan)','Profesor_Layton_Y_El_Futuro_Perdido_(SPA)','Sonic Colors','Mario vs. Donkey Kong_ Mini-Land Mayhem!','Apollo Justice Ace Attorney','El Profesor Layton y el futuro perdido','El Profesor Layton y la Caja de Pandora (ESP).NDS','El Profesor Layton y la Llamada Del Espectro (ESP) (Banenky)','El Profesor Layton y La LLamada del Espectro (FIX)(E)','El Profesor Layton y la llamada del espectro','El Profesor Layton Y La Villa Misteriosa','El_Profesor_Layton_y_la_llamada_del_espectro','Layton 1','Layton 2','Layton 3','Layton 4','Phoenix Wright - Ace Attorney (Italy) (WiiWare)','Phoenix Wright - Ace Attorney - Justice for All (Italy) (WiiWare)','Phoenix Wright - Ace Attorney - Trials and Tribulations (Italy) (WiiWare)','Phoenix Wright -Ace Attorney- (E)','Phoenix Wright Ace Attorney 3 Trials and Tribulations','Phoenix Wright Ace Attorney Justice For All','Phoenix Wright','Phoenix_Tribals_Tribulations','Profesor Layton Y El Futuro Perdido [NDS][EUR][ESP]','profesor layton y el futuro perdido','Profesor Layton y la Caja de Pandora','Professor Layton PROPER','Puzzle_Kingdoms','Time Hollow'];


In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from time import sleep
import pymysql



Lista_de_juegos = [
    "https://www.3djuegos.com/juegos/dead-space-remake/",
    "https://www.3djuegos.com/juegos/kirbys-return-to-dream-land-deluxe/"
]


try:
    connection = pymysql.connect(host='localhost',user='root',password='Kojinanjo1@',db='pelis')
    print("Conectado correctamente")
except pymysql.Error as e:
    print("No puede conectar con MySQL %d: %s" %(e.args[0], e.args[1]))


cursor = connection.cursor()

   

print("CONTINUE")

titulolista = list()
plataformaslista = list()
generoslista = list()
descripcionlista = list()
imagenlista = list()

df = pd.DataFrame()

for url in Lista_de_juegos:
    
    #print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    titulo = soup.find("div", attrs = {"id" : "gs_resumen"}).find("h1").text.strip()
    titulolista.append(titulo)
    plataformasElementos = soup.find_all("span", {"class": "cab_plat_off"})
    plataformas = []
    for plataforma in plataformasElementos:
        plataformas.append(plataforma.text.strip())
    plataformasString = ','.join(plataformas)
    plataformaslista.append(plataformasString)
    
    descripcion = soup.find("p", {"class": "s14 lh20 c2 mar_t15 mar_b10 fftit edit_desc"}).text.strip()
    descripcionlista.append(descripcion)
    
    geberoElementos = soup.find("dt", {"class": "edit_tematicas"}).find_next_sibling("dd").find_all("a")
    
    generos = []
    for gen in geberoElementos:
        generos.append(gen.text.strip())
    generosString = ','.join(generos)
    generoslista.append(generosString)
    
    imagen = soup.find("img", {"class": "dib"})
    imagenlista.append(imagen)
    

    sleep(3)
    
    #--------------------------------------------------------------------------------------------------------------------

df["titulo"] = titulolista
df["plataformas"] = plataformaslista
df["generos"] = generoslista
df["descripcion"] = descripcionlista
df["imagen"] = imagenlista

sql = """INSERT INTO `pelis`.`juegos` (`titulo`,`plataformas`,`generos`,`descripcion`,`imagen`) VALUES (%s, %s, %s, %s, %s)"""

for ind in df.index:
    cursor.execute(sql, (df['titulo'][ind], df['plataformas'][ind], df['generos'][ind], df['descripcion'][ind], df['imagen'][ind] ))
    connection.commit() 